In [ ]:
from openai import OpenAI

Basic Set ups

In [ ]:
# send request
# Create the OpenAI client
client = OpenAI(api_key="<OPENAI_API_TOKEN>")

# Create a request to the Chat Completions endpoint
response = client.chat.completions.create(
  # Specify the correct model
  model="gpt-4o-mini",      ## -- this model returns the most possible result
  # Specify the message keys
  messages=[{"role": "user", "content": "Who developed ChatGPT?"}]
)


# Usually wrapped up as "get_response" function
def get_response(message):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": message}]
    )
    return response.choices[0].message['content']

In [ ]:
# extracting from response

# extract the model used
print(response.model)

# extract the text from response
print(response.choices[0].message.content)

In [ ]:
# Temperature parameter: control determinism
# 0 = deterministic ~ 2 = random

# max_tokens: change length of response
# will increase cost!

Zero-shot: no example provided for the model to learn
One-shot, few-shot  - as indicated by the names

You can share examples and add classifications after "//"

Multi-turn tasks (chat competions)
- role "system" controls how "assistant" helps the "user". Controls assistant's behavior

- Can provide examples to guide the response. 
    = method 1: In-text learning - by including {"role":"assistant", "content":"example content"} in messages
    = method 2: Store response - create chat history, do back-and-forth convo

In [ ]:
# example of "Store response":
client = OpenAI(api_key="<OPENAI_API_TOKEN>")

messages = [{"role": "system", "content": "You are a helpful math tutor."}]
user_msgs = ["Explain what pi is.", "Summarize this in two bullet points."]

for q in user_msgs:
    print("User: ", q)
    
    # Create a dictionary for the user message from q and append to messages
    user_dict = {"role": 'user', "content": q}
    messages.append(user_dict)
    
    # Create the API request
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages = messages,
        max_tokens=100
    )
    
    # Convert the assistant's message to a dict and append to messages
    assistant_dict = {"role": 'assistant', "content": response.choices[0].message.content}
    messages.append(assistant_dict)
    print("Assistant: ", response.choices[0].message.content, "\n")

model = 'text-moderation-lastest" 
Allows you to do text moderation tasks (censorship)

In [ ]:
client = OpenAI(api_key="<OPENAI_API_TOKEN>")

# Create a request to the Moderation endpoint
response = client.moderations.create(
    model = 'text-moderation-latest',
    input ='My favorite book is To Kill a Mockingbird'
)

# Print the category scores
print(response.model_dump())

Whisper: speech-to-text

In [ ]:
# transcription request
response = client.audio.transcriptions.create(model="whisper-1", file = input_audio_file)

In [ ]:
# transcribing & translating at the same time
# only support English tho (at this point)
prompt = "Use less filler word. This audio is about X industry."
response = client.audio.translations.create(model="whisper-1", file = input_audio_file, prompt=prompt)

# can provide examples tell the model about ur preferences
# e.g. less filler word

Combining models

1 - Chaining. Feeding output from one model into another

    = can be same model
    = can be a different one


In [ ]:
client = OpenAI(api_key="<OPENAI_API_TOKEN>")

# Open the audio.wav file
audio_file = open("audio.wav", "rb")

# Create a transcription request using audio_file
audio_response = client.audio.transcriptions.create(
    model='whisper-1',
    file=audio_file
)

transcript = audio_response.text
message = "Tell me what language was used in this text: "+ transcript

# Create a request to the API to identify the language spoken
chat_response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages = [
        {"role":'user', 'content':message}]
)
print(chat_response.choices[0].message.content)

One-shot & Few-shot questions
provide examples for the model to learn. The more complex the task, the more examples you ought to provide

Multi-step prompts
- for sequential and cognitive tasks
Give steps gpt should follow. Nothing special..
(proactive)

Chain-of-thought prompts
Ask the model to generate intermediate thoughts
(reactive)
Risk: one flawed step leads to unsuccessful outcome

Self-consistency prompting
Generate multiple Chain-of-thought prompts by prompting the model several times
Determined by majority voting
"Provide 3 diffrent method in reasoning while answering this question. Final answer is obtained by majority vote"




### Use OpenAI for chatbot developments
- system prompt: define the purpose + specify the bot's role (it then knows how to answer 'who are you') + able to provide domain-specified knowledge
-- advanced: specify tone, audience, length, and structure
-- behavior guidance: conditional prompts. "If question in this field, answer at your best. If not, say 'idk'"

In [ ]:
client = OpenAI(api_key="<OPENAI_API_TOKEN>")

# Define the purpose of the chatbot
chatbot_purpose = "You're a customer support chatbot for an e-commerce company specializing in electronics. You'll assist users with inquiries, order tracking, and troubleshooting common issues. "

# Define audience guidelines
audience_guidelines = "The audience are tech-savvy individuals interested in purchasing electronic gadgets."

# Define tone guidelines
tone_guidelines = "Use a professional and user-friendly tone"

system_prompt = chatbot_purpose + audience_guidelines + tone_guidelines
response = get_response(system_prompt, "My new headphones aren't connecting to my device")
print(response)